In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import missingno as msnco

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/test.csv')

In [3]:
sample_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/sample_submission.csv')

In [4]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN


In [5]:
train_df.nunique()

ID                          1000000
Month                            12
Day_of_Month                     31
Estimated_Departure_Time       1365
Estimated_Arrival_Time         1428
Cancelled                         1
Diverted                          1
Origin_Airport                  374
Origin_Airport_ID               374
Origin_State                     52
Destination_Airport             375
Destination_Airport_ID          375
Destination_State                52
Distance                       1597
Airline                          28
Carrier_Code(IATA)               11
Carrier_ID(DOT)                  28
Tail_Number                    6430
Delay                             2
dtype: int64

In [6]:
train_df.drop(['Cancelled','Diverted'], axis=1, inplace= True) #Cancelled와 Diverted 데이터는 모든 데이터가 0으로 존재 --> drop 

In [ ]:
#Origin_Airport와 Origin_Airpirt_ID는 동일한 정보를 나타내므로 둘 중 하나를 drop
train_df.drop('Origin_Airport', axis=1, inplace= True)

In [ ]:
#Destination_Airport와 Destination_Airport_ID는 동일한 정보를 나타내므로 둘 중 하나 drop
train_df.drop('Destination_Airport', axis =1, inplace= True)

In [7]:
train_df.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

In [8]:
train_df.nunique() #airline과 carrier_ID(dot) 같은 정보 

ID                          1000000
Month                            12
Day_of_Month                     31
Estimated_Departure_Time       1365
Estimated_Arrival_Time         1428
Origin_Airport                  374
Origin_Airport_ID               374
Origin_State                     52
Destination_Airport             375
Destination_Airport_ID          375
Destination_State                52
Distance                       1597
Airline                          28
Carrier_Code(IATA)               11
Carrier_ID(DOT)                  28
Tail_Number                    6430
Delay                             2
dtype: int64

In [9]:
#delay 1, 0 으로 변환
dic = {'Not_Delayed' : 0, 'Delayed' : 1}
train_df['Delay'] = train_df['Delay'].map(dic) # delay float타입으로 변경

In [ ]:
train_df.columns

Index(['ID', 'Month', 'Day_of_Month', 'Estimated_Departure_Time',
       'Estimated_Arrival_Time', 'Origin_Airport_ID', 'Origin_State',
       'Destination_Airport_ID', 'Destination_State', 'Distance', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number', 'Delay'],
      dtype='object')

In [10]:
df = train_df[['ID','Estimated_Departure_Time','Estimated_Arrival_Time','Origin_State','Destination_State','Distance','Delay']]

In [11]:
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0,NaN
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN
...,...,...,...,...,...,...,...
999995,TRAIN_999995,936.0,1243.0,NaN,NaN,678.0,NaN
999996,TRAIN_999996,920.0,1028.0,NaN,Minnesota,223.0,NaN
999997,TRAIN_999997,800.0,1340.0,NaN,Texas,1642.0,NaN
999998,TRAIN_999998,1613.0,1824.0,Tennessee,NaN,214.0,NaN


#첫번째 비행시간 계산 

In [ ]:
train_df

In [ ]:
train_df['flight_time'] = train_df['Estimated_Arrival_Time']- train_df['Estimated_Departure_Time']  #비행시간 컬럼 추가 
train_df['flight_time'] = train_df['flight_time'].apply(lambda x : x+2400 if x<0 else x)
train_df['flight_time'] = train_df['flight_time'].apply(lambda x : x+40 if x%100//10>=6 else x)

In [ ]:
#첫번째 방법 -- 거리별 비행시간 계산
# import math

flight_mean = train_df['flight_time'].mean() #비행시간 평균
distance_mean =  train_df['Distance'].mean() #거리 평균
means = flight_mean/distance_mean

train_df['flight_time'] = train_df['flight_time'].fillna(round(train_df['Distance']*means))  #비행시간 결측치인 데이터에 거리별 비행시간 계산해서 넣기 


0.3175785343381852

In [ ]:
#두번째 방법  -- 거리별 비행시간을 회귀 모델로 학습시키기 
#x 거리  y 비행시간

In [ ]:
X_train = train_df['flight_time'].dropna()
idx = X_train.index

y_train = train_df.loc[idx]['flight_time']

X_train = X_train.values.reshape(-1,1) 
y_train = y_train.values.reshape(-1,1)

test_data = train_df[train_df['flight_time'].isna()] 
X_test = test_data['Distance']

X_test = X_test.values.reshape(-1,1)

In [ ]:
#단순 선형 회귀모델

# 모델 임포트 
from sklearn.linear_model import LinearRegression

# 모델 인스턴스 생성
lin_reg = LinearRegression()

# 모델 피팅
lin_reg.fit(X_train,y_train)

# 모델 예측
y_pred = lin_reg.predict(X_test)
y_pred



array([[ 419.],
       [ 199.],
       [ 369.],
       ...,
       [ 646.],
       [ 802.],
       [1476.]])

In [ ]:
#문제점 

# 시간을 빼서 구할 순 있음. 하지만 며칠이 걸렸는지가 알수 없어 애매. 

도착예정시간 - 출발예정시간 = 비행시간   
문제점 
- 정확한 도착 예정 날짜가 나와있는게 아닌 시간만 나와있어서 비행시간 25시간과 1시간의 차이를 구분할 수 없음. 
- 출발시간과 도착시간이 같아 비행시간이 0으로 나오는 경우가 존재 --> 출발예정시간은 출발주의 시간을, 도착예정시간은 도착주의 시간을 기준으로 나와있어 시차가 존재하여 단순히 도착예정시간과 출발예정시간을 빼면 안됨. 시차 고려가 필요  

해결방법  
- 출발주 A, 도착주가 B인 경우의 비행시간과  출발주B, 도착주A인 경우의 비행시간과  비교하여 각각 나온 비행시간을 빼고 2로 나누어 시차를 계산  


# 두번째 

In [ ]:
train_df[train_df['Origin_State']=='Texas']
#텍사스 출발 캘리포니아 도착 2041  2210   거리 1379   / 1시간 30분

In [ ]:
train_df[train_df['Origin_State']=='California'] 
#캘리포티아 출발 텍사스 도착 1150   1705  거리 1346  / 5시간 15분   

In [ ]:
train_df.nunique()

ID                          1000000
Month                            12
Day_of_Month                     31
Estimated_Departure_Time       1365
Estimated_Arrival_Time         1428
Origin_Airport                  374
Origin_Airport_ID               374
Origin_State                     52
Destination_Airport             375
Destination_Airport_ID          375
Destination_State                52
Distance                       1597
Airline                          28
Carrier_Code(IATA)               11
Carrier_ID(DOT)                  28
Tail_Number                    6430
Delay                             2
dtype: int64

In [ ]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN


In [ ]:
df = train_df[['ID','Estimated_Departure_Time','Estimated_Arrival_Time','Origin_State','Destination_State','Distance']]
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0
4,TRAIN_000004,900.0,1019.0,California,California,250.0
...,...,...,...,...,...,...
999995,TRAIN_999995,936.0,1243.0,NaN,NaN,678.0
999996,TRAIN_999996,920.0,1028.0,NaN,Minnesota,223.0
999997,TRAIN_999997,800.0,1340.0,NaN,Texas,1642.0
999998,TRAIN_999998,1613.0,1824.0,Tennessee,NaN,214.0


In [ ]:
df['Origin_State'].nunique

<bound method IndexOpsMixin.nunique of 0               Oklahoma
1               Illinois
2         North Carolina
3             California
4             California
               ...      
999995               NaN
999996               NaN
999997               NaN
999998         Tennessee
999999           Florida
Name: Origin_State, Length: 1000000, dtype: object>

In [ ]:
# for i in #origin_state.nuique  출발주 52개 리스트
    # for j in destination_state: 도착주 52개 리스트 
        # if df[(df['Origin_State']==i) & (df['Destination_State']==j)]
df = df.dropna(subset=['Origin_State','Destination_State'])

In [ ]:
df  # 시차를 알아 / 출발시간이 중요할 수도 있기 때문에 한 state를 기준으로 하는게 아니라 출발 state를 기준으로 시차를 다 적어주어야함

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0
4,TRAIN_000004,900.0,1019.0,California,California,250.0
...,...,...,...,...,...,...
999990,TRAIN_999990,NaN,751.0,Illinois,Texas,802.0
999991,TRAIN_999991,1720.0,1838.0,Virginia,New York,250.0
999992,TRAIN_999992,1307.0,1707.0,Wyoming,Illinois,1162.0
999993,TRAIN_999993,603.0,755.0,Virginia,Colorado,1452.0


In [ ]:
origin_state_list = df['Origin_State'].unique().tolist()
origin_state_list

In [ ]:
destination_state_list = df['Destination_State'].unique().tolist()
destination_state_list

In [ ]:
origin_state_list.sort()
destination_state_list.sort()

In [ ]:
origin_state_list == destination_state_list  #일치 

True

In [ ]:
df = df.dropna()
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0
4,TRAIN_000004,900.0,1019.0,California,California,250.0
6,TRAIN_000006,1742.0,1903.0,New Jersey,Massachusetts,200.0
...,...,...,...,...,...,...
999989,TRAIN_999989,1717.0,2019.0,New Jersey,Florida,1023.0
999991,TRAIN_999991,1720.0,1838.0,Virginia,New York,250.0
999992,TRAIN_999992,1307.0,1707.0,Wyoming,Illinois,1162.0
999993,TRAIN_999993,603.0,755.0,Virginia,Colorado,1452.0


In [ ]:
len(df)

630076

In [ ]:
li = origin_state_list
li

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'U.S. Pacific Trust Territories and Possessions',
 'U.S. Virgin Islands',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [ ]:
a = df[df['Origin_State']=='Alabama']['Destination_State'].values.tolist()

In [ ]:
a = list(set(a))
a

['North Carolina',
 'Virginia',
 'Colorado',
 'Illinois',
 'New York',
 'Maryland',
 'Florida',
 'Nevada',
 'Pennsylvania',
 'Michigan',
 'Texas',
 'Georgia']

In [ ]:
for i in a:
    if df[df['Origin_State']==i]

In [ ]:
df[(df['Origin_State']=='Virginia') & (df['Destination_State']=='Alabama')]  #이렇게 해서 평균 거리랑, 도착시간-출발시간 //  alabama 출발 virginia 도착 // 1시간 10분 

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance
3049,TRAIN_003049,750.0,908.0,Virginia,Alabama,693.0
6494,TRAIN_006494,837.0,945.0,Virginia,Alabama,653.0
12711,TRAIN_012711,1420.0,1527.0,Virginia,Alabama,613.0
16134,TRAIN_016134,759.0,908.0,Virginia,Alabama,693.0
19627,TRAIN_019627,1705.0,1808.0,Virginia,Alabama,596.0
...,...,...,...,...,...,...
985137,TRAIN_985137,1705.0,1807.0,Virginia,Alabama,596.0
986352,TRAIN_986352,815.0,910.0,Virginia,Alabama,596.0
993629,TRAIN_993629,758.0,922.0,Virginia,Alabama,693.0
997056,TRAIN_997056,758.0,913.0,Virginia,Alabama,693.0


In [ ]:
df[(df['Origin_State']=='Alabama') & (df['Destination_State']=='Virginia')]  #평균 거리랑 도착시간- 출발시간  ///  3시간   --> 시차 한시간 차이 맞네 

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance
110,TRAIN_000110,940.0,1247.0,Alabama,Virginia,693.0
2773,TRAIN_002773,1057.0,1355.0,Alabama,Virginia,653.0
4846,TRAIN_004846,1111.0,1359.0,Alabama,Virginia,613.0
15118,TRAIN_015118,603.0,906.0,Alabama,Virginia,653.0
18033,TRAIN_018033,1112.0,1418.0,Alabama,Virginia,653.0
...,...,...,...,...,...,...
977535,TRAIN_977535,1056.0,1359.0,Alabama,Virginia,653.0
981121,TRAIN_981121,901.0,1159.0,Alabama,Virginia,613.0
989382,TRAIN_989382,855.0,1150.0,Alabama,Virginia,596.0
993554,TRAIN_993554,1829.0,2130.0,Alabama,Virginia,613.0


In [ ]:
#함수로 만들어서 최종 데이터프레임에 apply 하는 식으로?


# 시차 고려

In [ ]:
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0,NaN
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN
...,...,...,...,...,...,...,...
999990,TRAIN_999990,NaN,751.0,Illinois,Texas,802.0,NaN
999991,TRAIN_999991,1720.0,1838.0,Virginia,New York,250.0,NaN
999992,TRAIN_999992,1307.0,1707.0,Wyoming,Illinois,1162.0,0.0
999993,TRAIN_999993,603.0,755.0,Virginia,Colorado,1452.0,NaN


In [ ]:
def find_time_lag(train_df, state):
    OA = train_df[train_df['Origin_State'] == state]
    predict_st = OA.Destination_State.unique().tolist()
    DA = train_df[train_df['Destination_State'] == state]
    time_delay_dict = {}
    for st in predict_st:
        st_oa = OA[OA['Destination_State'] == st]
        st_oa_diff = st_oa.Estimated_Arrival_Time.mean() - st_oa.Estimated_Departure_Time.mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = st_da.Estimated_Arrival_Time.mean() - st_da.Estimated_Departure_Time.mean()
        time_delay = st_da_diff - st_oa_diff
        time_delay_dict[state, st] = -((time_delay/2)//60)*60
    
    return time_delay_dict

In [ ]:
time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))

In [ ]:
time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))
print(time_delay)

{('Oklahoma', 'Texas'): 0, ('Oklahoma', 'Colorado'): -2, ('Oklahoma', 'Georgia'): 1, ('Oklahoma', 'Illinois'): 0, ('Oklahoma', 'New Jersey'): 1, ('Oklahoma', 'California'): -3, ('Oklahoma', 'Virginia'): 1, ('Oklahoma', 'Florida'): 1, ('Oklahoma', 'Missouri'): 0, ('Oklahoma', 'Nevada'): -1, ('Oklahoma', 'North Carolina'): 2, ('Oklahoma', 'Minnesota'): 0, ('Oklahoma', 'Arizona'): -2, ('Oklahoma', 'Utah'): -2, ('Oklahoma', 'Maryland'): 1, ('Oklahoma', 'Michigan'): 1, ('Oklahoma', 'Pennsylvania'): 1, ('Oklahoma', 'Washington'): -3, ('Illinois', 'Utah'): -1, ('Illinois', 'Missouri'): 0, ('Illinois', 'Kentucky'): 0, ('Illinois', 'Tennessee'): 0, ('Illinois', 'Ohio'): 0, ('Illinois', 'Minnesota'): 0, ('Illinois', 'West Virginia'): 2, ('Illinois', 'Michigan'): 1, ('Illinois', 'Oklahoma'): 0, ('Illinois', 'Washington'): 0, ('Illinois', 'New York'): 1, ('Illinois', 'Texas'): 0, ('Illinois', 'California'): 0, ('Illinois', 'Indiana'): 1, ('Illinois', 'Illinois'): 0, ('Illinois', 'Pennsylvania'): 0

In [ ]:
len(time_delay)

1398

In [ ]:
s = pd.DataFrame(time_delay, index=['time_delay']).T

In [ ]:
def find_time_lag(train_df, state):
    OA = train_df[train_df['Origin_State'] == state]  #출발 state a
    predict_st = OA.Destination_State.unique().tolist() #출발 state a의 도착 state 리스트 //  destination_list와 동일 
    DA = train_df[train_df['Destination_State'] == state] #도착 state a
    time_delay_dict = {}
    for st in predict_st:
        st_oa = OA[OA['Destination_State'] == st] #출발a , 도착[1,2,3,4]  for문 
        st_oa_diff = st_oa.Estimated_Arrival_Time.mean() - st_oa.Estimated_Departure_Time.mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = st_da.Estimated_Arrival_Time.mean() - st_da.Estimated_Departure_Time.mean()
        time_delay = st_da_diff - st_oa_diff
        if not np.isnan(time_delay):
            time_delay_dict[state, st] = -round(time_delay/2/60)
    
    return time_delay_dict

In [ ]:
train_df.isna().sum()

In [ ]:
test_df.isna().sum()

# 세번째 

In [ ]:
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0,NaN
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN
...,...,...,...,...,...,...,...
999995,TRAIN_999995,936.0,1243.0,NaN,NaN,678.0,NaN
999996,TRAIN_999996,920.0,1028.0,NaN,Minnesota,223.0,NaN
999997,TRAIN_999997,800.0,1340.0,NaN,Texas,1642.0,NaN
999998,TRAIN_999998,1613.0,1824.0,Tennessee,NaN,214.0,NaN


In [ ]:
df.isna().sum()

ID                               0
Estimated_Departure_Time     86628
Estimated_Arrival_Time       86516
Origin_State                     0
Destination_State                0
Distance                         0
Delay                       591429
dtype: int64

In [ ]:
df = df.dropna(subset=['Origin_State','Destination_State']) #결측치 제거 

In [ ]:
state_list = df['Origin_State'].unique().tolist()
state_list

In [ ]:
list(set(df[df['Origin_State']=='Alabama']['Destination_State'].values.tolist())) #Alabama를 입력해주는 값에 따라 달라지기 ~

['Illinois',
 'North Carolina',
 'Georgia',
 'Florida',
 'Texas',
 'New York',
 'Virginia',
 'Nevada',
 'Michigan',
 'Pennsylvania',
 'Maryland',
 'Colorado']

In [ ]:
#과정
# 1 . orinin_state == a / a는 전체 52개 state중 하나 
# 2. a에서 출발/ 도착한 state b -- b는 list(set(df[df['Origin_State']=='a']['Destination_State'].values.tolist())) 이 리스트에서 하나 
# 3 . 출발주 a / 도착주 b -- 여기서 도착시간 - 출발시간 의 평균 --  첫번째 시간 A_time  
# 4. 출발 b / 도착 a -- 여기서 도착시간- 출발시간의 평균 --  두번째 시간  B_time
# 5 . (A_time - B_time)/2 

In [ ]:
#1. origin_state ==a 
df[df['Origin_State']=='Alabama']

#2. a(Alabama)에서 출발하고 도착한 주 리스트로 정리
destination_list = list(set(df[df['Origin_State']=='Alabama']['Destination_State'].values.tolist()))

#3. 출발주a / 도착주 b-- 위에 정리한 리스트 중 하나 
for b in destination_list:
    df[(df['Origin_State']=='Alabama') & (df['Destination_State']==b)]

df[(df['Origin_State']=='Alabama') & (df['Destination_State']=='Illinois')]
df[(df['Origin_State']=='Illinois') & (df['Destination_State']=='Alabama')]

In [ ]:
df[df['Origin_State']=='Alabama']

In [ ]:
destination_list = list(set(df[df['Origin_State']=='Alabama']['Destination_State'].values.tolist()))
destination_list

In [ ]:
a = df[(df['Origin_State']=='Alabama') & (df['Destination_State']=='Illinois')]
a_time = (a['Estimated_Arrival_Time']-a['Estimated_Departure_Time']).mean()  #1번 

In [ ]:
b = df[(df['Origin_State']=='Illinois') & (df['Destination_State']=='Alabama')]
b_time = (b['Estimated_Arrival_Time']-b['Estimated_Departure_Time']).mean()  #2번 
# b.isna().sum()

In [ ]:
#최종 
# 1번 - 2번 /2 
(a_time - b_time)/2

9.706989529712502

In [ ]:
destination_list #state = ;Alibama

['Illinois',
 'North Carolina',
 'Georgia',
 'Florida',
 'Texas',
 'New York',
 'Virginia',
 'Nevada',
 'Michigan',
 'Pennsylvania',
 'Maryland',
 'Colorado']

In [ ]:
def time_diff(state):
    destination_list =  list(set(df[df['Origin_State']==state]['Destination_State'].values.tolist()))
    for d in destination_list:
        a = df[(df['Origin_State']==state) & (df['Destination_State']==d)] #ali 출발 , texas 도착
        b = df[(df['Origin_State']==d) & (df['Destination_State']==state)] #texas 출. ali 도착
        a_time = (a['Estimated_Arrival_Time']-a['Estimated_Departure_Time']).mean()     
        b_time = (b['Estimated_Arrival_Time']-b['Estimated_Departure_Time']).mean()
        time_di = (a_time - b_time)/2 #a_state와 b_state 의 시차  --> 출발a 도착b or 출발b 도착a인 데이터에 시차 컬럼 넣어주기 
        
        #1)dict 형태로 출발지:도착지 = 시차  ---> ?
    return     

In [ ]:
#시차 까지 구했는데 이 시차를 데이터 프레임 안에 time_diff 컬럼에 새로 넣어야함
# for 문 한번 돌때ㅔ 마다 . state와 d (destination_list 원소 하나)의 시차를 구하게 되는데 이 시차를, 출발state도착d & 출발d도착state 인 데이터에 넣어야함


In [ ]:
time_diff('Alabama')

9.706989529712502

In [ ]:
data = df.head()
data

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0,NaN
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN


In [ ]:
data['diff_time'] = data.apply(lambda x : x['Estimation_Departure_Time'] )

In [ ]:
#여기서 차이를 타임 diff로 넣는다고 하면
data['diff_time'].apply(lambda x : )

ValueError: ignored

In [ ]:
#1 ) 비행시간 > 시차계산 > 시차를 계산한 비행시간 ex) 도착-출발=3시간, 시차 1시간, 비행시간 = 4시간 > 거리별 비행시간을 계산하여 결측치 채워넣기  /// 여기서 이상치 계산하기  
#2 ) 

In [ ]:
df[(df['Origin_State']=='Virginia') & (df['Destination_State']=='Alabama')]  #이렇게 해서 평균 거리랑, 도착시간-출발시간 //  alabama 출발 virginia 도착 // 1시간 10분 

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance
3049,TRAIN_003049,750.0,908.0,Virginia,Alabama,693.0
6494,TRAIN_006494,837.0,945.0,Virginia,Alabama,653.0
12711,TRAIN_012711,1420.0,1527.0,Virginia,Alabama,613.0
16134,TRAIN_016134,759.0,908.0,Virginia,Alabama,693.0
19627,TRAIN_019627,1705.0,1808.0,Virginia,Alabama,596.0
...,...,...,...,...,...,...
985137,TRAIN_985137,1705.0,1807.0,Virginia,Alabama,596.0
986352,TRAIN_986352,815.0,910.0,Virginia,Alabama,596.0
993629,TRAIN_993629,758.0,922.0,Virginia,Alabama,693.0
997056,TRAIN_997056,758.0,913.0,Virginia,Alabama,693.0


In [ ]:
df[(df['Origin_State']=='Alabama') & (df['Destination_State']=='Virginia')]  #평균 거리랑 도착시간- 출발시간  ///  3시간   --> 시차 한시간 차이 맞네 

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance
110,TRAIN_000110,940.0,1247.0,Alabama,Virginia,693.0
2773,TRAIN_002773,1057.0,1355.0,Alabama,Virginia,653.0
4846,TRAIN_004846,1111.0,1359.0,Alabama,Virginia,613.0
15118,TRAIN_015118,603.0,906.0,Alabama,Virginia,653.0
18033,TRAIN_018033,1112.0,1418.0,Alabama,Virginia,653.0
...,...,...,...,...,...,...
977535,TRAIN_977535,1056.0,1359.0,Alabama,Virginia,653.0
981121,TRAIN_981121,901.0,1159.0,Alabama,Virginia,613.0
989382,TRAIN_989382,855.0,1150.0,Alabama,Virginia,596.0
993554,TRAIN_993554,1829.0,2130.0,Alabama,Virginia,613.0
